In [ ]:
'''
This script loads output data from control_task_ISC.py run
on the Discovery cluster and runs various analyses depending
on user input at the top of the script. See the "set
parameters" chunk below for descriptions of the various
analyses.
'''

In [3]:
import scipy.io as sio
import os
import numpy as np
import pandas as pd
import time
from joblib import Parallel, delayed
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as multi
from scipy.stats import norm
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.cm as cm
import warnings
import matplotlib.cbook
import pickle
import sys
from scipy.interpolate import interp1d
sys.path.append('/dartfs-hpc/rc/home/z/f00589z/hyperscanning/support_scripts/')
from phaseScramble import *
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation) # suppress some matplotlib warnings
%matplotlib inline

In [4]:
from platform import python_version

print(python_version())

3.9.4


In [5]:
baseFolder = '/dartfs-hpc/rc/home/z/f00589z/hyperscanning/control_tasks/'
loadFolder = baseFolder + 'control_ISC_output/'
inputFolder = baseFolder + 'nuisRegr_output_files/'

In [6]:
# select parameters of dataset to load
permutations = 5000 # number of permutations
cShift = True # cShift (True) or pScram (False)
normalize = True # time series z-scale normalized before ISC
alpha = 0.05 # alpha for permutation tests
twoTailed = True # two (True) or one-tailed (False; right tailed) permutation tests applied
useDetrendedData = True # use data that was detrended during nilearn nuisance regression step
fitNormal = False # load data that includes null distribution normal fit parameters
removeListeningSamples = 0
removeReadingSamples = 0

# set analyses to include
plotCorrDists = False # plot ISC distributions for each subject for each control task
plotMinMaxMedianISCtimeSeries = False # look at sub vs group time series in voxels with min, max, and median ISC coefficients
subBySubCorrMats = False # plot mean correlation values across voxels between subs
plotPairwiseISCtimeSeries = False # NOTE that subBySubCorrMats must be True for this to run
analyzeSmoothness = False # compute and plot time series smoothness measure (this was a concern early on but has largely been obviated as of June 2021)
analyzeDrift = False # compute and plot time series drift measure
findOptimalDriftWindow = False # find the shortest initial time windows to remove from each task to minimize drift ('analyzeDrift' must be True)
ISC_statMap = True # plot median ISC heatmaps on an average brain surface
drift_statMap = True # plot mean drift on an average brain surface
smooth_statMap = False # plot mean smoothness on an average brain surface

In [7]:
# get file name based on input above
fileName = 'controlISC_' + str(permutations) + 'perm'
if cShift:
    fileName = fileName + '_cShift'
else:
    fileName = fileName + '_pScram'
if normalize:
    fileName = fileName + '_norm'
if twoTailed:
    fileName = fileName + '_twoTailed'
if useDetrendedData:
    fileName = fileName + '_detrended'
    epiTag = 'detrended_'
else:
    epiTag = ''
if fitNormal:
    fileName = fileName + '_nullDistFits'
if removeListeningSamples > 0:
    fileName = fileName + '_xL' + str(removeListeningSamples)
if removeReadingSamples > 0:
    fileName = fileName + '_xR' + str(removeReadingSamples)

# load data
with open(loadFolder + fileName + '.pkl', 'rb') as f:
    permTest, corrData, groupFitData, duration, pGroup = pickle.load(f)
    print('loaded file: ' + loadFolder + fileName + '.pkl')

loaded file: /dartfs-hpc/rc/home/z/f00589z/hyperscanning/control_tasks/control_ISC_output/controlISC_5000perm_cShift_norm_twoTailed_detrended.pkl


In [8]:
# load hyperscanning subject list
subList = pd.read_pickle('/dartfs-hpc/rc/home/z/f00589z/hyperscanning/misc/hyperscanning_subject_list.pkl')

# get number of participants
numSubs = subList.shape[0]

# get number of pairs
numPairs = round(numSubs / 2)

# get number of voxels (using the first subject's ISC data from the listening task)
numVox = len(corrData[0][0])

# define condition labels
taskNames = ['listening','reading']
siteNames = ['DBIC','CBS']

# indicate that we have not loaded the EPI time series
epiLoaded = False

# colorblind-friendly colors list
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

# set task colors
taskColors = CB_color_cycle[:2]

In [12]:
with open('/dartfs-hpc/rc/home/z/f00589z/hyperscanning/misc/hyperscanning_subject_list.pkl','wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([subList], f, protocol=1)

In [17]:
subList.to_csv('/dartfs-hpc/rc/home/z/f00589z/hyperscanning/misc/hyperscanning_subject_list.csv',index=False)

In [14]:
s.getcwd()

'/dartfs-hpc/rc/home/z/f00589z'

In [ ]:
def loadEPI(subList,folder,normalize,epiTag):

    """
    function for loading EPI time series
    :param subList: subs x 3 dataframe with columns: 'pairNum', 'site', and 'subID'
    :param folder: folder from which to load EPI time series
    :param normalize: boolean indicating whether or not to normalize time series
    :return: boldData: time series as numpy arrays
    """

    numS = subList.shape[0]
    taskNames = ['listening','reading']

    # loop through participants...
    boldData = [[]] * 2
    for TASK in [0,1]: #for each task, listening, then reading

        # preallocate task data list
        boldData[TASK] = [[]] * numS

        for S in range(numS):

            # get file name
            file = folder + 'sub-' + str(subList['subID'][S]) + '_ses-pair0' + str(subList['pairNum'][S]) + '_task-storytelling' + str(TASK + 3) + '_run-0' + str(TASK + 3) + '_bold_space-MNI152NLin2009cAsym_preproc_nuisRegr_2021_' + epiTag + 'interp.mat'

            # load data
            tmp = sio.loadmat(file) #load file
            boldData[TASK][S] = tmp['tseries'] #get timeseries data
            print('loaded ' + str(boldData[TASK][S].shape[0]) + ' x ' + str(boldData[TASK][S].shape[1]) + ' timeseries for ' + taskNames[TASK] + ' task, sub ' + subList['subID'][S])

            if normalize:
                boldData[TASK][S] = stats.zscore(boldData[TASK][S],axis=0)
                print('z-scoring time series')

    return boldData

In [ ]:
"""
This is effectively here just in case you want
clarity on how the multi.fdrcorrection function
works. You verified that the function below
gives the same output.
"""

def fdr(pVals,q):

    # get p values
    pVals = np.sort(pVals)

    # get "unsorting" indices so you can map the hypothesis testing results back to the proper voxels
    unsortInds = pVals.argsort().argsort()

    # find threshold
    N = len(pVals)
    i = np.arange(1, N+1) # the 1-based i index of the p values, as in p(i)

    # print number of uncorrected -values below q
    print('# uncorrected pVals < ' + str(q) + ': ' + str(len(np.where(pVals < q)[0])))

    # get pVals below qi / N
    below = pVals < (q * i / N)

    # if any critical value exists
    if np.where(below)[0].size > 0:

        # get index (sorted) of greatest pVal below qi / N
        max_below = np.max(np.where(below)[0])

        # get FDR adjusted p value
        pCrit = pVals[max_below]

        # print number of uncorrected -values below q
        print('# uncorrected pVals < ' + str(pCrit) + ': ' + str(len(np.where(pVals <= pCrit)[0])))

        # hypothesis test
        h = (pVals <= pCrit)[unsortInds]

    else:

        h = np.zeros(N, dtype=bool)

    return h

In [ ]:
def surfaceStatMap(masker,statMapVec,avgSurface,thresh):

    # preallocate task arrays
    statMap = [[]] * 2
    texture = [[]] * 2
    view = [[]] * 2

    # for each task...
    for TASK in [0,1]:

        # get stat map
        statMap[TASK] = masker.inverse_transform(statMapVec[TASK])

        # surface plot
        texture[TASK] = [[]] * 2
        view[TASK] = [[]] * 2

        for HEMI in [0,1]:
            if HEMI == 0:
                texture[TASK][HEMI] = surface.vol_to_surf(statMap[TASK], avgSurface.pial_left)
                view[TASK][HEMI] = plotting.view_surf(avgSurface.infl_left,
                                                                       texture[TASK][HEMI],
                                                                       threshold=thresh,
                                                                       colorbar=True,
                                                                       title= taskNames[TASK] + ', left',
                                                                       bg_map=avgSurface.sulc_left)
            else:
                texture[TASK][HEMI] = surface.vol_to_surf(statMap[TASK], avgSurface.pial_right)
                view[TASK][HEMI] = plotting.view_surf(avgSurface.infl_right,
                                                                       texture[TASK][HEMI],
                                                                       threshold=thresh,
                                                                       colorbar=True,
                                                                       title=taskNames[TASK] + ', right',
                                                                       bg_map=avgSurface.sulc_right)

    return view

In [ ]:
if plotCorrDists:

    # set axis label font size
    axLabFontSize = 12

    # plot data
    for SUB in range(numPairs): # for each pair...

        # get subjects from current pair
        pairSubs = [SUB,SUB + numPairs]

        # initialize plot
        plt.figure(facecolor='white',figsize=(6,6))

        # for each subject in the current pair
        for PAIRSUB in [0,1]:

            for TASK in [0,1]:

                # get plot data
                pData = corrData[TASK][pairSubs[PAIRSUB]]

                # select subplot
                plt.subplot(2, 2, PAIRSUB*2 + TASK + 1)

                # plot histogram
                plt.hist(pData, bins=25, density=True, alpha=0.6, color=taskColors[TASK])

                # dashed line at x=0
                yMax = plt.gca().get_ylim()[1]
                plt.plot([0, 0], [0, yMax], '--k')

                # axes and title
                plt.xlabel('correlation', fontsize=axLabFontSize)
                if TASK == 0:
                    plt.ylabel('voxel count', fontsize=axLabFontSize)
                plt.title(taskNames[TASK] + ', sub ' + siteNames[PAIRSUB] + str(SUB + 1))

        plt.tight_layout()
        plt.show()

In [ ]:
"""
For each participant and each task we plot the distribution
of ISC coefficients (correlation between the participant's
voxel time series and the mean voxel time series among all
of the other participants) across voxels. We then overlay
the individual voxel time series from the participant and
the rest of the group that have the minimum, maximum, and
median ISC coefficients across voxels.
"""

if plotMinMaxMedianISCtimeSeries:

    # load EPI time series if necessary
    if not epiLoaded:
        boldData = loadEPI(subList,inputFolder,normalize,epiTag)
        epiLoaded = True # indicate that we've loaded the EPI time series

    # extreme voxel labels
    voxLabs = ['min corr vox','max corr vox','median vox']
    voxColors = ['y','m','k']

    # set task colors
    taskColors = CB_color_cycle[:2]

    # make subplotting map
    spMap3 = np.arange(8).reshape(4,2) + 1

    # set axis label font size
    axLabFontSize = 12

    # define standard scaler
    scaler = StandardScaler()

    # plot data
    for SUB in range(numSubs):

        # initialize plot
        plt.figure(facecolor='white',figsize=(16,8))

        # set main plot title
        titleString = subList['subID'][SUB]
        plt.suptitle(titleString)

        for TASK in [0,1]:

            # get plot data
            pData = corrData[TASK][SUB]

            # select subplot for histogram
            plt.subplot(spMap3.shape[0], spMap3.shape[1], spMap3[0,TASK])

            # plot histogram
            plt.hist(pData, bins=100, density=True, alpha=0.6, color=taskColors[TASK])

            # dashed line at x=0
            yMax = plt.gca().get_ylim()[1]
            plt.plot([0, 0], [0, yMax], '--k')

            # axes and title
            plt.xlabel('correlation', fontsize=axLabFontSize)
            if TASK == 0:
                plt.ylabel('voxel count', fontsize=axLabFontSize)
            plt.title(taskNames[TASK])

            # plot voxel time series with extreme values
            for VOX in [0,1,2]: # min, max, median

                # get "Extreme Index" of voxel with either min or max value (or median)
                if VOX == 0:
                    EIND = np.unravel_index(np.argmin(pData),pData.shape) # minimum correlation voxel index
                elif VOX == 1:
                    EIND = np.unravel_index(np.argmax(pData),pData.shape) # maximum correlation voxel index
                elif VOX == 2:
                    EIND = np.argsort(pData)[len(pData)//2] # median (approximately)

                # add locations of min and max correlation to histogram for reference
                extremeCorr = pData[EIND]
                plt.subplot(spMap3.shape[0], spMap3.shape[1], spMap3[0,TASK])
                plt.plot([extremeCorr, extremeCorr], [0, yMax], '-' + voxColors[VOX])

                # get individual subject time series at the extreme voxel
                y1 = boldData[TASK][SUB][:,EIND]
                x = np.array(range(len(y1))) + 1

                # get mean of data from all participants EXCEPT the current participant
                otherSubs = np.arange(0,numSubs)
                otherSubs = np.delete(otherSubs,SUB)
                y2 = np.mean([boldData[TASK][i][:,EIND] for i in otherSubs], axis=0)
                if VOX == 2: #hack to deal with EIND not being a tuple when we find the median
                    y2 = y2.reshape(y2.shape[0],1)
                y2 = scaler.fit_transform(y2) # normalize the rest-of-group mean (see next section for confirmation that this doesn't influence correlations)

                # select subplot and reset subplot border color
                ax = plt.subplot(spMap3.shape[0], spMap3.shape[1], spMap3[VOX + 1,TASK])
                plt.setp(ax.spines.values(), color=voxColors[VOX])
                plt.setp([ax.get_xticklines(), ax.get_yticklines()], color=voxColors[VOX])

                # plot lines and add legend
                line1, = plt.plot(x,y1,'-k',label = 'individual')
                line2, = plt.plot(x,y2,'-', label = 'rest of group', color = taskColors[TASK]) # , linewidth=2
                plt.legend(handles=[line1, line2],loc='upper right')

                if TASK == 0:
                    plt.xlabel('TR')
                else:
                    plt.xlabel('reading stimulus flip')
                plt.ylabel('BOLD signal')
                plt.title(voxLabs[VOX])

        plt.tight_layout()
        plt.show()

In [ ]:
# from matplotlib.colors import Normalize
if subBySubCorrMats:

    # load EPI time series if necessary
    if not epiLoaded:
        boldData = loadEPI(subList,inputFolder,normalize,epiTag)
        epiLoaded = True # indicate that we've loaded the EPI time series

    corrMat = [[]] * 2
    corrColors = [[]] * 2
    corrData_pairs = [[]] * 2
    axLab = [[]] * numSubs

    for TASK in [0,1]:

        corrMat[TASK] = [[]] * 2
        corrColors[TASK]= [[]] * 2
        corrData_pairs[TASK]= [[]] * 2

        # some feedback
        print('\ncomputing pairwise correlations for ' + str(taskNames[TASK]) + ' task')

        # preallocate subs x subs correlation matrix
        corrMat[TASK] = np.empty([numSubs,numSubs])
        corrData_pairs[TASK] = [[]] * numSubs

        for SUB1 in range(numSubs):

            corrData_pairs[TASK][SUB1] = [[]] * numSubs

            # get axis labels
            if TASK == 0:
                if SUB1 < numPairs:
                    axLab[SUB1] = 'D' + str(SUB1 + 1)
                else:
                    axLab[SUB1] = 'H' + str(SUB1 - numPairs + 1)

            # set the diagonal equal to 1
            corrMat[TASK][SUB1,SUB1] = 1

            for SUB2 in np.arange(SUB1 + 1,numSubs):

                corrData_pairs[TASK][SUB1][SUB2] = fastColumnCorr(boldData[TASK][SUB1], boldData[TASK][SUB2])
                corrMat[TASK][SUB1,SUB2] = np.mean(corrData_pairs[TASK][SUB1][SUB2])

                #fill in the other half of corrMat so the plots dont look weird
                corrMat[TASK][SUB2,SUB1] = corrMat[TASK][SUB1,SUB2]

        plt.figure(facecolor='white')
        cmap = cm.get_cmap('RdBu')#sns.diverging_palette(20, 220, n=200)
        ax = sns.heatmap(
            corrMat[TASK],
            vmin=-1, vmax=1, center=0,
            cmap=cmap,
            square=True
        )
        ax.set_xticklabels(axLab)
        ax.set_xticklabels(
            ax.get_xticklabels(),
            rotation=45,
            horizontalalignment='right'
        )
        ax.set_yticklabels(axLab)
        ax.set_yticklabels(
            ax.get_yticklabels(),
            rotation=0
        )

        # add a title
        plt.title('mean corr coef across vox, ' + taskNames[TASK] + ' task')

        # get heatmap rgbs
        im = ax.collections[0]
        corrColors[TASK] = im.cmap(im.norm(im.get_array()))

    # pairwise version of individual voxel time series comparisons
    if plotPairwiseISCtimeSeries:

        # make a numSubs by numSubs plot map
        spMap4 = np.arange(numSubs**2).reshape(numSubs,numSubs)

        # set plot width [inches?]
        plotWidth = 16

        # plot data
        for SUB1 in range(numSubs):

            # get sub1 string
            if SUB1 < numPairs:
                sub1Str = 'D' + str(SUB1 + 1)
            else:
                sub1Str = 'H' + str(SUB1 - numPairs + 1)

            for SUB2 in np.arange(SUB1 + 1,numSubs):

                # get sub2 string
                if SUB2 < numPairs:
                    sub2Str = 'D' + str(SUB2 + 1)
                else:
                    sub2Str = 'H' + str(SUB2 - numPairs + 1)

                # initialize plot
                plt.figure(facecolor='white',figsize=(16,8))

                # main title
                plt.suptitle('subs ' + sub1Str + ' & ' + sub2Str)

                for TASK in [0,1]:

                    # get correlation data for a given pair
                    pData = corrData_pairs[TASK][SUB1][SUB2]

                    # plot histogram
                    plt.subplot(spMap3.shape[0], spMap3.shape[1], spMap[0,TASK])
                    plt.hist(pData, bins=100, density=True, alpha=0.6, color=taskColors[TASK])

                    # dashed line at x=0
                    yMax = plt.gca().get_ylim()[1]
                    plt.plot([0, 0], [0, yMax], '--k')

                    # axes and title
                    plt.xlabel('correlation', fontsize=axLabFontSize)
                    if TASK == 0:
                        plt.ylabel('voxel count', fontsize=axLabFontSize)
                    plt.title(taskNames[TASK])

                    for VOX in [0,1,2]: # min, max, median

                        # get "Extreme Index" of voxel with either min or max value (or median)
                        if VOX == 0:
                            EIND = np.unravel_index(np.argmin(pData),pData.shape) # minimum correlation voxel index
                        elif VOX == 1:
                            EIND = np.unravel_index(np.argmax(pData),pData.shape) # maximum correlation voxel index
                        elif VOX == 2:
                            EIND = np.argsort(pData)[len(pData)//2] # median (approximately)

                        # add locations of min and max correlation to histogram for reference
                        extremeCorr = pData[EIND]
                        plt.subplot(spMap3.shape[0], spMap3.shape[1], spMap3[0,TASK])
                        plt.plot([extremeCorr, extremeCorr], [0, yMax], '-', color=voxColors[VOX])

                        # get individual subject time series at the extreme voxel
                        y1 = boldData[TASK][SUB1][:,EIND]
                        y2 = boldData[TASK][SUB2][:,EIND]
                        x = np.array(range(len(y1))) + 1

                        # select subplot for time series line plot
                        ax = plt.subplot(spMap3.shape[0], spMap3.shape[1], spMap3[VOX + 1,TASK])
                        plt.setp(ax.spines.values(), color=voxColors[VOX])
                        plt.setp([ax.get_xticklines(), ax.get_yticklines()], color=voxColors[VOX])

                        line1, = plt.plot(x,y1,'-k',label = sub1Str)
                        line2, = plt.plot(x,y2,'-', label = sub2Str, color = taskColors[TASK])
                        plt.legend(handles=[line1, line2],loc='upper right')

                        if TASK == 0:
                            plt.xlabel('TR')
                        else:
                            plt.xlabel('reading stimulus flip')
                        plt.ylabel('BOLD signal')
                        plt.title(voxLabs[VOX])

                # display plots
                plt.tight_layout()
                plt.show()

In [ ]:
"""
Here we use a basic formula for quantifying smoothness:
sd(diff(x))/abs(mean(diff(x)))
Adapted the formula from here:
https://stats.stackexchange.com/questions/24607/how-to-measure-smoothness-of-a-time-series-in-r
The inversion is so that large values reflect greater
smoothness. Smoothness values can then be optionally
standardized from 0 to 1 to make them a bit more
interpratable.

For each subject and each task we plot the distribution
of smoothness values across voxels -- only showing those
voxels less than or equal to the __th percentile, because
these distributions are deeply right skewed. We then plot
the individual time series for voxels at the min and max
and three selected percentile values for smoothness.

"""

if analyzeSmoothness:

    # function to find nearest value to a given percentile in an array
    def find_nearest_percentile_index(array, percentile):
        array = np.asarray(array)
        target = np.percentile(array, percentile)
        idx = (np.abs(array - target)).argmin()
        return idx

    # set up a subplot map
    spMap = np.arange(6).reshape(3,2) + 1

    # preallocate task arrays for smoothness
    smoothness = [[]] * 2

    # option to standardize smoothness values
    standardize0to1 = True

    # select colors for different individual voxels
    voxColors = CB_color_cycle[4:9]

    # threshold percentile below which to include data for histogram (deals with extreme skew)
    threshPerc = 90

    # select smoothness percentiles to look at in case voxMethod == 'percentile' above
    percentiles = [25, 50, threshPerc]

    # make voxel labels
    if standardize0to1:
        smoothLabs = ['min smoothness = 0',str(percentiles[0]) + ' %', str(percentiles[1]) + ' %', str(percentiles[2]) + ' %','max smoothness = 1']
    else:
        smoothLabs = ['min smoothness',str(percentiles[0]) + ' %', str(percentiles[1]) + ' %', str(percentiles[2]) + ' %','max smoothness']

    # get and plot smoothness values
    for TASK in [0,1]: # for each task...

        # preallocate sub arrays for smoothness
        smoothness[TASK] = [[]] * numSubs

        for SUB in range(numSubs): # for each subject...

            # initialize plot
            plt.figure(facecolor='white',figsize=(16,8))

            # main title (subID)
            plt.suptitle(taskNames[TASK] + ' sub ' + str(SUB + 1))

            # get data
            data = boldData[TASK][SUB]

            # compute smoothness
            smoothness[TASK][SUB] = 1 / np.std(np.diff(data,axis=0),axis=0) / abs(np.mean(np.diff(data,axis=0),axis=0)) # see description above for source of formula

            # optional z-score standardization
            if standardize0to1:
                smoothness[TASK][SUB] = (smoothness[TASK][SUB] - np.min(smoothness[TASK][SUB])) / (np.max(smoothness[TASK][SUB]) - np.min(smoothness[TASK][SUB]))

            # arbitrarily subset for plotability (because these are so skewed)
            data = smoothness[TASK][SUB]

            # get voxel indices for time series with various levels of smoothness smoothness
            evox = [[]] * 5
            evox[0] = np.unravel_index(np.argmin(data),data.shape)[0]
            counter = 1
            for PERC in percentiles:
                evox[counter] = find_nearest_percentile_index(data, PERC)
                counter += 1
            evox[4] = np.unravel_index(np.argmax(data),data.shape)[0]

            # select subplot for histogram
            plt.subplot(spMap.shape[0], spMap.shape[1], 1)

            # select subset of data to plot for histogram to deal with visualization problems from extreme skew
            threshInd = find_nearest_percentile_index(data, threshPerc)
            plotData = data[data <= data[threshInd]]

            # plot smoothness histogram
            plt.hist(plotData, bins=100, density=True, alpha=1, color=taskColors[TASK])
            plt.xlabel('smoothness parameter')
            plt.ylabel('density')
            if standardize0to1:
                plt.title('standardized (0 to 1) smoothness values up to ' + str(threshPerc) + ' percentile')
            else:
                plt.title('smoothness values up to ' + str(threshPerc) + ' percentile')

            # get histogram max y-value
            yMax = plt.gca().get_ylim()[1]

            # plot single voxel timeseries
            for VOX in range(len(evox)):

                # add vertical bars to histogram
                if data[evox[VOX]] <= data[threshInd]:
                    plt.subplot(spMap.shape[0], spMap.shape[1], 1)
                    smoothVal = data[evox[VOX]]
                    plt.plot([smoothVal, smoothVal], [0, yMax], '-', color=voxColors[VOX])

                # get time series at the extreme voxel
                y = boldData[TASK][SUB][:,evox[VOX]]
                x = np.array(range(len(y))) + 1

                # select subplot for time series line plot
                ax = plt.subplot(spMap.shape[0], spMap.shape[1], VOX + 2)
                plt.setp(ax.spines.values(), color=voxColors[VOX])
                plt.setp([ax.get_xticklines(), ax.get_yticklines()], color=voxColors[VOX])

                # plot time series
                plt.plot(x,y,'-k')

                # subplot title and axis labels
                plt.title(smoothLabs[VOX])
                if TASK == 0:
                    plt.xlabel('TR')
                else:
                    plt.xlabel('reading stimulus flip')
                plt.ylabel('BOLD signal')

            plt.tight_layout()
            plt.show()

    ###############################################
    ### get smoothness summary measures (means) ###
    ###############################################

    # preallocate
    smoothnessCons = [[]] * 2
    smoothness_mean = [[]] * 2

    # get mean drift measure across subs
    for TASK in [0,1]:

        smoothnessCons[TASK] = np.empty([numSubs,numVox])

        for SUB in range(numSubs):

            # make sure everything is standardized
            smoothnessCons[TASK][SUB,:] = (smoothness[TASK][SUB] - np.nanmean(smoothness[TASK][SUB])) / np.std(smoothness[TASK][SUB])

        smoothness_mean[TASK]  = np.nanmean(smoothnessCons[TASK], axis=0)

In [ ]:
"""
compare the mean signal of an early epoch to that of a late epoch. Greater absolute differences
should indicate greater drift. NOTE that this is super hacky, but should be FAST and at least
somewhat sensitive

set ending and starting time points for the early and late epochs for each task, respectively
epochBorders = [[10,5],[100,50]] would mean...
early epochs for the listening and reading tasks would be time points 1-10 and 1-5, respectively
and the late epochs would be 100-end, 50-end, also respectively
"""

if analyzeDrift:

    # define epoch borders
    epochBorders = [[10,5],[100,50]]

    # subplot map
    spMap = np.arange(6).reshape(3,2) + 1

    # set percentiles, colors, labels
    voxColors = CB_color_cycle[4:9]
    percentiles = [10, 50, 90]
    sdSf = 2 # standard deviation scaling factor
    voxMethod = 'stdevs'
    if voxMethod == 'stdevs':
        diffLabs = ['most negative diff','mean - 1SD*' + str(sdSf), 'mean','mean + 1SD*' + str(sdSf) ,'most negative diff']
    else:
        diffLabs = ['most negative diff',str(percentiles[0]) + ' %', str(percentiles[1]) + ' %', str(percentiles[2]) + ' %','most positive diff']

    # standardize difference scores
    stdDiff = True

    # preallocate arrays
    epoch = [[]] * 2
    driftHack = [[]] * 2

    # load EPI time series if necessary
    if not epiLoaded:
        boldData = loadEPI(subList,inputFolder,normalize,epiTag)
        epiLoaded = True # indicate that we've loaded the EPI time series

    # get number of samples in the time series from each task, using the normalized data from the first subject
    numSamps = [boldData[0][0].shape[0], boldData[1][0].shape[0]]

    for TASK in [0,1]:

        # get epoch time points
        epoch[TASK] = [[]] * 2 # preallocate
        epoch[TASK][0] = np.arange(epochBorders[0][TASK]) # early epoch
        lateEpochWidth = numSamps[TASK] - epochBorders[1][TASK] + 1
        epoch[TASK][1] = np.arange(lateEpochWidth) + epochBorders[1][TASK] - 1

        # preallocate
        driftHack[TASK] = [[]] * numSubs

        for SUB in range(numSubs):

            # initialize plot
            plt.figure(facecolor='white',figsize=(16,8))

            # main title
            plt.suptitle(taskNames[TASK] + ' sub ' + str(SUB + 1))

            # get time series for current sub
            data = boldData[TASK][SUB]

            # compute hacky drift statistic
            driftHack[TASK][SUB] = np.mean(data[tuple(epoch[TASK][0]),:],axis=0) - np.mean(data[tuple(epoch[TASK][1]),:],axis=0)

            # optional standardization
            if stdDiff:
                driftHack[TASK][SUB] = (driftHack[TASK][SUB] - np.mean(driftHack[TASK][SUB])) / np.std(driftHack[TASK][SUB])

            # select subplot for histogram
            plt.subplot(spMap.shape[0], spMap.shape[1], 1)

            # plot difference histogram
            plt.hist(driftHack[TASK][SUB], bins=100, density=True, alpha=0.5, color=taskColors[TASK])
            plt.xlabel('mean(first ' + str(epochBorders[0][TASK]) + ' time points) - mean(last ' + str(numSamps[TASK] - epochBorders[1][TASK] + 1) + ' timpoints')
            plt.ylabel('proportion of voxels')

            # get voxel indices for time series with min and max difference scores and those at various percentile cutoffs
            evox = [[]] * 5
            evox[0] = np.unravel_index(np.argmin(driftHack[TASK][SUB]),driftHack[TASK][SUB].shape)[0]
            if voxMethod == 'stdevs':
                evox[1] = (np.abs(driftHack[TASK][SUB] - (np.mean(driftHack[TASK][SUB]) - np.std(driftHack[TASK][SUB]) * sdSf))).argmin()
                evox[2] = (np.abs(driftHack[TASK][SUB] - np.mean(driftHack[TASK][SUB]))).argmin()
                evox[3] = (np.abs(driftHack[TASK][SUB] - (np.mean(driftHack[TASK][SUB]) + np.std(driftHack[TASK][SUB]) * sdSf))).argmin()
            else:
                counter = 1
                for PERC in percentiles:
                    evox[counter] = find_nearest_percentile_index(driftHack[TASK][SUB], PERC)
                    counter += 1
            evox[4] = np.unravel_index(np.argmax(driftHack[TASK][SUB]),driftHack[TASK][SUB].shape)[0]

            # get histogram max y-value
            yMax = plt.gca().get_ylim()[1]

            # plot single voxel timeseries
            for VOX in range(len(evox)):

                # add vertical bars to histogram
                plt.subplot(spMap.shape[0], spMap.shape[1], 1)
                diffVal = driftHack[TASK][SUB][evox[VOX]]
                plt.plot([diffVal, diffVal], [0, yMax], '-', color=voxColors[VOX])

                # get time series at the extreme voxel
                y = boldData[TASK][SUB][:,evox[VOX]]
                x = np.array(range(len(y))) + 1

                # select subplot for time series line plot
                ax = plt.subplot(spMap.shape[0], spMap.shape[1], VOX + 2)
                plt.setp(ax.spines.values(), color=voxColors[VOX])
                plt.setp([ax.get_xticklines(), ax.get_yticklines()], color=voxColors[VOX])

                # plot time series
                plt.plot(x,y,'-k')

                # subplot title and axis labels
                plt.title(diffLabs[VOX])
                if TASK == 0:
                    plt.xlabel('TR')
                else:
                    plt.xlabel('reading stimulus flip')
                plt.ylabel('BOLD signal')

            plt.tight_layout()
            plt.show()

    ##########################################
    ### get drift summary measures (means) ###
    ##########################################

    # preallocate
    driftHackCons = [[]] * 2
    driftHack_mean = [[]] * 2

    # get mean drift measure across subs
    for TASK in [0,1]:

        # preallocate
        driftHackCons[TASK] = np.empty([numSubs,numVox])

        for SUB in range(numSubs):

            # make sure everything is standardized
            driftHackCons[TASK][SUB,:] = (driftHack[TASK][SUB] - np.nanmean(driftHack[TASK][SUB])) / np.std(driftHack[TASK][SUB])

        # get mean drift
        driftHack_mean[TASK] = np.nanmean(driftHackCons[TASK], axis=0)

In [ ]:
if findOptimalDriftWindow:

    # load EPI time series if necessary
    if not epiLoaded:
        boldData = loadEPI(subList,inputFolder,normalize,epiTag)
        epiLoaded = True # indicate that we've loaded the EPI time series

    # set plotting scheme
    individPlots = True
    groupPlots = True

    # set epoch widths and get the maximum number of TRs to remove
    widths = [3,6] # epoch widths [TRs]
    removalMax = 100

    # get subplotting scheme
    pRows = np.ceil(len(widths) / 2)
    if len(widths) == 1:
        pCols = 1
    else:
        pCols = 2

    # standardize difference scores
    stdDiff = True

    # preallocate arrays
    meanAbsDrift = [[]] * 2

    # drift threshold scaling factor (to scale by 1 SD) - any voxels
    # with drift values less than this distance from the mean drift
    # value will be ignored in the analysis below. If set to zero,
    # no thresholding will be applied.
    threshSF = 2

    for TASK in [0,1]: # for each task...

        # preallocate
        meanAbsDrift[TASK] = [[]] * numSubs

        for SUB in range(numSubs): # for each subject...

            # get time series for current sub
            data = boldData[TASK][SUB]

            # preallocate
            meanAbsDrift[TASK][SUB] = np.empty([len(widths),removalMax])

            # initialize individual plot and set title
            if individPlots:
                plt.figure(facecolor='white',figsize=(pCols * 4,pRows * 4))
                if threshSF > 0:
                    plt.suptitle(taskNames[TASK] + ' task, sub ' + str(SUB + 1) + ', drift threshold: +/-' + str(threshSF) + 'SD')
                else:
                    plt.suptitle(taskNames[TASK] + ' task, sub ' + str(SUB + 1) + ', no drift thresholding')

            for WIDTH in range(len(widths)): # for each epoch width...

                # feedback
                print('\nanalyzing ' + taskNames[TASK] + ' sub ' + str(SUB + 1) + ' width ' + str(WIDTH + 1))

                for TRX in range(removalMax): # for each number of TRs removed

                    # remove TRX TRs
                    if TRX > 0:
                        data = np.delete(data,0,0)

                    # get epochs
                    epochs = [np.arange(widths[WIDTH]), np.arange(widths[WIDTH],data.shape[0])]

                    # compute drift statistic
                    drift = np.mean(data[tuple(epochs[0]),:],axis=0) - np.mean(data[tuple(epochs[1]),:],axis=0)
                    if TRX == 0: # if thresholding, select the voxels with the greatest drift prior to removing TRs
                        if threshSF > 0:
                            mu = np.mean(drift)
                            sd = np.std(drift)
                            thresholds = [mu - sd * threshSF, mu + sd * threshSF]
                            voxInds = np.concatenate((np.argwhere(drift < thresholds[0]), np.argwhere(drift > thresholds[1])))
                        else:
                            voxInds = range(numVox)
                    meanAbsDrift[TASK][SUB][WIDTH,TRX] = np.mean(np.abs(drift[voxInds]))

                # optional standardization across TR removals for each width
                if stdDiff:
                    meanAbsDrift[TASK][SUB][WIDTH,:] = (meanAbsDrift[TASK][SUB][WIDTH,:] - np.mean(meanAbsDrift[TASK][SUB][WIDTH,:])) / np.std(meanAbsDrift[TASK][SUB][WIDTH,:])

                # individual plots
                if individPlots:
                    plt.subplot(pRows,pCols,WIDTH+1)
                    plt.plot(range(removalMax),meanAbsDrift[TASK][SUB][WIDTH,:],'-ok')
                    plt.xlabel('# samples removed',fontsize=16)
                    plt.ylabel('mean absolute drift',fontsize=16)
                    plt.title('epoch width = ' + str(widths[WIDTH]) + ' samples',fontsize=16)

            # clean setup for individual plots
            if individPlots:
                plt.tight_layout()
                plt.show()

    # preallocate group stats arrays
    groupMeanAbsDrift = [[]] * 2
    groupSDAbsDrift = [[]] * 2

    # get / plot group stats
    driftSamps = [17,10] # hardcoding the thresholds for now
    for TASK in [0,1]: # for each task...

        # compute group mean drift
        groupMeanAbsDrift[TASK] = np.mean([meanAbsDrift[TASK][i] for i in range(numSubs)], axis=0)
        groupSDAbsDrift[TASK] = np.std([meanAbsDrift[TASK][i] for i in range(numSubs)], axis=0)

        # plot group mean drift
        if groupPlots:

            # initialize plot
            plt.figure(facecolor='white',figsize=(pCols * 4,pRows * 4))
            if threshSF > 0:
                plt.suptitle(taskNames[TASK] + ' task, group mean absolute drift (N=' + str(numSubs) + '), drift threshold: +/-' + str(threshSF) + 'SD')
            else:
                plt.suptitle(taskNames[TASK] + ' task, group mean absolute drift (N=' + str(numSubs) + '), no drift thresholding')

            for WIDTH in range(len(widths)):
                plt.subplot(pRows,pCols,WIDTH+1)
                x = range(removalMax)
                y = groupMeanAbsDrift[TASK][WIDTH,:]
                error = groupSDAbsDrift[TASK][WIDTH,:]
                plt.plot(x, y, 'k-')
                plt.fill_between(x, y-error, y+error)
                plt.xlabel('# samples removed',fontsize=16)
                plt.ylabel('mean absolute drift',fontsize=16)
                plt.title('epoch width = ' + str(widths[WIDTH]) + ' samples',fontsize=16)

                # estimate the "elbow" of the group mean curve
                # fit an exponential curve
                popt, pcov = curve_fit(func, x, y, p0=(1, 1e-6, -1))
                x2 = np.linspace(np.min(x),np.max(x),100)
                y2 = func(x2, *popt)
                plt.plot(x2,y2,'--r',linewidth=2)

                yLims = plt.gca().get_ylim()
                plt.plot([driftSamps[TASK],driftSamps[TASK]],yLims,'-r')

            plt.tight_layout()
            plt.show()

In [ ]:
"""
#############################################
########### Note about stat maps! ###########
#############################################

Currently plotting stat maps by generating a 'view' list
then printing its subcomponents in successive notebook
chunks. Hence why the stat maps sections are broken into
so many chunks.
"""

In [ ]:
if ISC_statMap or ISC_statMap or drift_statMap:

    # import nilearn modules
    from nilearn import image as nImage
    from nilearn import input_data
    from nilearn import datasets
    from nilearn import surface
    from nilearn import plotting

    # get masker object
    maskFile = '/dartfs-hpc/rc/home/z/f00589z/hyperscanning/control_tasks/nuisRegr_input_files/mni_asym09c_mask_resamp3x3.nii.gz'
    maskImg = nImage.load_img(maskFile)
    masker = input_data.NiftiMasker(maskImg)
    masker.fit_transform(maskImg)

    mapDir = '/dartfs-hpc/rc/home/z/f00589z/hyperscanning/control_tasks/statMaps/'
    fsaverage = datasets.fetch_surf_fsaverage()

In [ ]:
if ISC_statMap:

    #########################################
    ### set hypothesis testing parameters ###
    #########################################

    # threshold by proportion of subjects with a significant FDR corrected p-value at a given voxel
    propThresh = False

    # if thresholding by the proportion of subjects with a significant FDR corrected p-value at a given voxel...
    if propThresh:

        # set proportion of participants who need to have fdr corrected
        # significant median ISC coefficients at a voxel to include it
        # in the mask
        fdrProp = .5

        # preallocate
        fdrVecs = [[]] * 2
        fdrMask = [[]] * 2

        for TASK in [0,1]:

            # preallocate
            fdrVecs[TASK] = np.empty([numSubs,numVox])
            fdrMask[TASK] = np.zeros([numVox,1])

            for SUB in range(numSubs):

                # compile fdr hypothesis testing vectors (1=reject null, 0=fail to reject null)
                fdrVecs[TASK][SUB,:] = permTest[TASK][SUB][1][0]

            # generate group mask based on fdrProp
            for VOX in range(numVox):

                if np.sum(fdrVecs[TASK][:,VOX]) > (numSubs * fdrProp):
                    fdrMask[TASK][VOX] = 1

    else:

        # evaluate real median ISC against null distribution of ISC group medians
        homeBrew = False
        alpha = 0.05

    # make an array from 0 to numSubs
    subNums = np.arange(0,numSubs)

    # preallocate task arrays for mean ISC coefficients
    corrData_median = [[]] * 2

    # for each task...
    for TASK in [0,1]:

        # get mean ISC across subs
        corrData_median[TASK] = np.median([corrData[TASK][i] for i in subNums], axis=0)

        if propThresh:
            corrData_median[TASK][fdrMask[TASK][:,0] == 0] = 0
        else:

            # get FDR hypothesis testing array
            if homeBrew:
                h = fdr(pGroup[TASK][METHOD][0], alphaPrime)
            else:
                # h = multi.fdrcorrection(pGroup[TASK][METHOD][0], alpha=alpha)[0]
                h = pGroup[TASK][1][0]
            print('\n' + str(len(np.where(h == True)[0])) + ' hits')

            # mask out voxels that failed to reject the null by setting them to 0
            corrData_median[TASK][h == False] = 0 # try NaN?

    # get surface plots
    thresh = 0.001 # threshold the stat maps just above zero so that voxels where null is not rejected are not plotted
    view = surfaceStatMap(masker,corrData_median,fsaverage,thresh)

In [ ]:
view[0][0]

In [ ]:
view[0][1]

In [ ]:
view[1][0]

In [ ]:
view[1][1]

In [ ]:
if smooth_statMap:
    if not analyzeSmoothness:
        print('\nYou need to run the "analyzeSmoothness" chunk before you can unlock this stat map.\n')
    else:
        thresh = 0.99
        view = surfaceStatMap(masker,smoothness_mean,fsaverage,thresh)

In [ ]:
if smooth_statMap:
    view[0][0]

In [ ]:
if smooth_statMap:
    view[0][1]

In [ ]:
if smooth_statMap:
    view[1][0]

In [ ]:
if smooth_statMap:
    view[1][1]

In [ ]:
if drift_statMap:
    if not analyzeDrift:
        print('\nYou need to run the "analyzeDrift" chunk before you can unlock this stat map.\n')
    else:
        thresh = 1
        view = surfaceStatMap(masker,driftHack_mean,fsaverage,thresh)

In [ ]:
if drift_statMap:
    view[0][0]

In [ ]:
if drift_statMap:
    view[0][1]

In [ ]:
if drift_statMap:
    view[1][0]

In [ ]:
if drift_statMap:
    view[1][1]


